In [ ]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")
  
  # Mount the Google Drive at mount
  mount='/content/drive'
  print("Colab: mounting Google drive on ", mount)

  drive.mount(mount)

  # Switch to the directory on the Google Drive that you want to use
  import os
  drive_root = mount + "/My Drive/Colab Notebooks/ML-for-IC-Die-Images"
  
  # Create drive_root if it doesn't exist
  create_drive_root = True
  if create_drive_root:
    print("\nColab: making sure ", drive_root, " exists.")
    os.makedirs(drive_root, exist_ok=True)
  
  # Change to the directory
  print("\nColab: Changing directory to ", drive_root)
  %cd $drive_root
  !pwd

  # !pip install -r requirements.txt
  # !sudo apt-get autoremove


  from IPython.display import JSON
  from google.colab import output
  from subprocess import getoutput
  
  # @title jQuery Terminal's [Features](https://terminal.jcubic.pl/)

  def shell(command):
    if command.startswith('cd'):
      path = command.strip().split(maxsplit=1)[1]
      os.chdir(path)
      return JSON([''])
    return JSON([getoutput(command)])
  output.register_callback('shell', shell)

In [ ]:
if IN_COLAB:
    !pip install cupy-cuda112 tensorflow-addons

In [ ]:
# %%
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import more_itertools
import sys
import cv2
import os
import time
from matplotlib import style
#style.use('classic')
from numpy import genfromtxt, asarray, savez_compressed, load
from sklearn.model_selection import train_test_split
import PIL
from PIL import Image 
from io import StringIO
# import memory_profiler
import numpy as np
import cupy
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import time
#style.use('classic')
from joblib import Parallel, delayed
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
from scipy.signal import resample
from io import StringIO
import os
import shutil
#plt.style.use('ggplot')
#matplotlib.use( 'tkagg' )
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pylab as pl
import seaborn as sns
sns.set_style("ticks",{'axes.grid' : True})
from pathlib import Path
from collections.abc import Iterable
import math


import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv3D,Activation,Conv2D,Cropping2D, ConvLSTM2D, MaxPooling2D, MaxPooling3D, BatchNormalization, Flatten, Input, Dense, GRU, Embedding, LSTM, SimpleRNN, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical, Sequence, normalize
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
from tensorflow.python.ops import math_ops, control_flow_ops
from tensorflow.python.framework import constant_op


if IN_COLAB:
  from google.colab import drive
  from tensorflow.keras.optimizers.legacy import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
  from tqdm.notebook import tqdm_notebook
else:
  from tensorflow.keras.optimizers import RMSprop,Adam,SGD,Adadelta,Adagrad,Adamax
  from tqdm.autonotebook import tqdm


from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()


ModuleNotFoundError: No module named 'more_itertools'

In [ ]:
from tensorflow.python.platform import build_info as tf_build_info
import tensorflow.python.platform.build_info as build
from tensorflow.compat.v1.keras.backend import set_session

try:
  # tf.debugging.experimental.enable_dump_debug_info('.', tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)
  # tf.debugging.set_log_device_placement(True)
  from tensorflow.python.client import device_lib

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  config.gpu_options.per_process_gpu_memory_fraction = 0.1
  try:
    tf.compat.v1.InteractiveSession.close()
  except Exception as error:
    print("Error trying to delete session")
    print(error) 
  sess = tf.compat.v1.InteractiveSession(config=config)
  set_session(sess)
  print(device_lib.list_local_devices())
  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
    try:
    
      tf.config.experimental.set_memory_growth(gpu, True)
      # Restrict TensorFlow to only use the first GPU
      tf.config.set_visible_devices(gpus[0], 'GPU')
      logical_gpus = tf.config.list_logical_devices('GPU')
      print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
      # Visible devices must be set before GPUs have been initialized
      print(e)

except Exception as error:
    print("Error trying to configure computing device.")
    print(error)

print("TensorFlow verison: ",tf.__version__)
print("CUDA verison: ", build.build_info['cuda_version'])
print("CUDNN verison: ", build.build_info['cudnn_version'])



### Set Tensorflow-GPU precision 
Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory. 

In [ ]:
# tf.keras.backend.floatx()
# tf.keras.backend.set_floatx('float16')
# tf.keras.backend.set_floatx('float32')
# policy = tf.keras.mixed_precision.Policy('mixed_float16')
# tf.keras.mixed_precision.set_global_policy(policy)
tf.keras.backend.floatx()

In [ ]:
# Change as you wish
if IN_COLAB:
  DATA_FOLDER = './data'
  TRAIN_FOLDER = './data/Train'
  VALIDATION_FOLDER = './data/Validation'
  IMAGE_INPUT_FOLDER = './data/Image_Input'
  IMAGE_OUTPUT_FOLDER = './data/Image_Output'
else:
  DATA_FOLDER = './../data'
  TRAIN_FOLDER = './../data/Train'
  VALIDATION_FOLDER = './../data/Validation'
  IMAGE_INPUT_FOLDER = './../data/Image_Input'
  IMAGE_OUTPUT_FOLDER = './../data/Image_Output'


In [ ]:
# Set the paths to your annotated image folders
input_image_path = os.path.join(IMAGE_INPUT_FOLDER)
output_image_path = os.path.join(IMAGE_OUTPUT_FOLDER)


In [ ]:
# Get the list of image and label file paths
input_filenames = sorted(glob.glob(os.path.join(input_image_path,'*.jpg')))[:]
output_filenames = sorted(glob.glob(os.path.join(output_image_path,'*.jpg')))[:]

len(input_filenames), len(output_filenames)

In [ ]:
for idx, inp_outp in enumerate(zip(input_filenames[:], output_filenames[:])):
  print(idx, inp_outp)

In [ ]:
if len(input_filenames[:])==len(output_filenames[:]):
  print('Number of files in Input and Output folders are equal')
else:
  min_file_length = min([len(input_filenames[:]),len(output_filenames[:])])
  print(f'Uneven number images in Input and Output folders! \nSelecting minimum possible number of images: {min_file_length}')

In [ ]:
min_file_length=-1
if IN_COLAB:
  check_ims = [in_name.split('/')[3]==out_name.split('/')[3] for in_name,out_name in zip(input_filenames[:min_file_length], output_filenames[:min_file_length])]
else:
  check_ims = [in_name.split('\\')[1]==out_name.split('\\')[1] for in_name,out_name in zip(input_filenames[:min_file_length], output_filenames[:min_file_length])]
any(not x for x in check_ims) #Check if at least one image names is not corresponding 

In [ ]:
test_size = 0.05  #Specify test percentage

# Split the image and label file paths into training and validation sets
train_image_paths, validation_image_paths, train_label_paths, validation_label_paths = train_test_split(input_filenames[:], output_filenames[:], test_size=test_size, shuffle=True)

# Create the training and validation folders
train_folder_path = os.path.join(TRAIN_FOLDER)
validation_folder_path = os.path.join(VALIDATION_FOLDER)

os.makedirs(os.path.join(train_folder_path, 'Input_Image'), exist_ok=True)
os.makedirs(os.path.join(train_folder_path, 'Output_Image'), exist_ok=True)
os.makedirs(os.path.join(validation_folder_path, 'Input_Image'), exist_ok=True)
os.makedirs(os.path.join(validation_folder_path, 'Output_Image'), exist_ok=True)

'''
# Copy the training images and labels to the training folder
for image_path, label_path in zip(train_image_paths, train_label_paths):
    image_name = os.path.basename(image_path)
    label_name = os.path.basename(label_path)
    shutil.copy(image_path, os.path.join(train_folder_path, 'Input_Image', image_name))
    shutil.copy(label_path, os.path.join(train_folder_path, 'Output_Image',label_name))

# Copy the validation images and labels to the validation folder
for image_path, label_path in zip(validation_image_paths, validation_label_paths):
    image_name = os.path.basename(image_path)
    label_name = os.path.basename(label_path)
    shutil.copy(image_path, os.path.join(validation_folder_path, 'Input_Image', image_name))
    shutil.copy(label_path, os.path.join(validation_folder_path, 'Output_Image', label_name))
'''

In [ ]:
class CustomDataGenerator(Sequence):
    def __init__(self, input_paths, output_paths, batch_size, input_shape, num_classes=5, plot=False):
        self.input_paths = input_paths
        self.output_paths = output_paths
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.plot = plot
        self.num_classes = num_classes

    def __len__(self):
        return int(np.ceil(len(self.input_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        input_batch_paths = self.input_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        output_batch_paths = self.output_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        input_images = cupy.asnumpy(list(map(self.image_dataloader, input_batch_paths))).astype(np.float32)
        output_images = cupy.asnumpy(list(map(self.image_dataloader, output_batch_paths))).astype(np.float32)
        segmentation_mask = cupy.asnumpy(list(map(self.mask_generator, [output_image - input_image for input_image, output_image in zip(input_images,output_images)]))).astype(np.float32)
        return input_images, segmentation_mask #.reshape(-1,height*width,channels)
        

    def image_dataloader(self, image_path):
        # Load the input image
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        # image = image.reshape(image.shape[0],image.shape[1],image.shape[2])
        image = image.reshape(image.shape[0],image.shape[1],1)
        # Resize the image with padding
        image = cv2.resize(image, self.input_shape, interpolation=cv2.INTER_AREA)
        # image = resize_with_padding(image, self.input_shape)
        # Add an alpha channel to the input image
        # image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
        if self.plot:
            plt.figure()
            plt.imshow(image)
            plt.title(f'Path: {image_path}')
            plt.show()
        return image.reshape(image.shape[0],image.shape[1],1)/255.
    
    def mask_generator(self, image):
        blurred_image = skimage.filters.gaussian(image, sigma=1.0)
        t = skimage.filters.threshold_otsu(blurred_image)
        binary_mask = blurred_image > t
        # selection = output_validation_batch[i]/255.
        # selection[~binary_mask] = 0
        return binary_mask
        
    def resize_with_padding(self, image, target_size):
        """
        Resizes an image with padding and maintains aspect ratio while keeping the background transparent.
        :param image: input image as a numpy array
        :param target_size: a tuple (width, height) representing the target size of the output image
        :return: the resized image as a numpy array
        """
        h, w = image.shape[:2]
        target_w, target_h = target_size

        # Calculate the scale factor and the new dimensions
        scale = min(target_w / w, target_h / h)
        new_w = int(w * scale)
        new_h = int(h * scale)

        # Resize the image
        image_resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        # Create a transparent background
        background = np.zeros((target_h, target_w, 4), dtype=np.uint8)
        background[:, :, 3] = 0  # set alpha channel to zero

        # Calculate the coordinates to paste the resized image
        x = int((target_w - new_w) / 2)
        y = int((target_h - new_h) / 2)

        # Paste the resized image onto the transparent background
        background[y:y+new_h, x:x+new_w, :3] = image_resized
        background[y:y+new_h, x:x+new_w, 3] = 255  # set alpha channel to 255 (fully opaque)

        return image_resized

In [ ]:
height, width, channels = 512, 512, 1   # Choose maximum possible image size in order not to compromise quality
batch_size = 3

In [ ]:
train_generator = CustomDataGenerator(train_image_paths, train_label_paths, batch_size, 
                                      input_shape=(height, width), num_classes=5, plot=False)

validation_generator = CustomDataGenerator(validation_image_paths, validation_label_paths, batch_size, 
                                           input_shape=(height, width), num_classes=5, plot=False)

input_train_batch, output_train_batch=next(iter(train_generator))

print('x_train shape: ', input_train_batch.shape, 'x_train dtype:', input_train_batch.dtype)
print('y_train shape: ', output_train_batch.shape, 'y_train dtype:', output_train_batch.dtype)
# print('y_train shape: ', output_train_batch[0].shape, 'y_train dtype:', output_train_batch[0].dtype)
# print('y_train shape: ', output_train_batch[1].shape, 'y_train dtype:', output_train_batch[1].dtype)

input_validation_batch, output_validation_batch=next(iter(validation_generator))

print('x_val shape: ', input_validation_batch.shape, 'x_val dtype:', input_validation_batch.dtype)  
print('y_val shape: ', output_validation_batch.shape, 'y_val dtype:', output_validation_batch.dtype)
# print('y_val shape: ', output_validation_batch[0].shape, 'y_val dtype:', output_validation_batch[0].dtype)
# print('y_val shape: ', output_validation_batch[1].shape, 'y_val dtype:', output_validation_batch[1].dtype)

In [ ]:
for i in range(3):
    
    blurred_image = skimage.filters.gaussian(output_validation_batch[i]/255., sigma=1.0)
    t = skimage.filters.threshold_otsu(blurred_image)
    binary_mask = blurred_image > t
    selection = output_validation_batch[i]/255.
    selection[~binary_mask] = 0



    import skimage
    # blur the image to denoise
    blurred_image = skimage.filters.gaussian(output_validation_batch[i]/255., sigma=1.0)

    # show the histogram of the blurred image
    histogram, bin_edges = np.histogram(blurred_image, bins=256, range=(0.0, 1.0))
    fig, ax = plt.subplots()
    plt.plot(bin_edges[0:-1], histogram)
    plt.title("Graylevel histogram")
    plt.xlabel("gray value")
    plt.ylabel("pixel count")
    plt.xlim(0, 1.0)

    # perform automatic thresholding
    t = skimage.filters.threshold_otsu(blurred_image)
    print("Found automatic threshold t = {}.".format(t))

    # create a binary mask with the threshold found by Otsu's method
    binary_mask = blurred_image > t

    fig, ax = plt.subplots()
    plt.imshow(binary_mask, cmap="gray")
    plt.show()

    # apply the binary mask to select the foreground
    selection = output_validation_batch[i]/255.
    selection[~binary_mask] = 0

    fig, ax = plt.subplots()
    plt.imshow(selection)
    plt.show()

    fig, ax = plt.subplots()
    plt.imshow(output_validation_batch[i]/255.)
    plt.show()

In [ ]:
plt.imshow(input_validation_batch[0], cmap='gray')
plt.title('Original Image')
plt.show()
plt.imshow(output_validation_batch[0], cmap='gray')
plt.title('Annotation Mask')
plt.show()
masked_image = input_validation_batch[0][~output_validation_batch[0].astype('bool').squeeze( axis=-1)]
plt.imshow(masked_image, cmap='gray')
plt.title('Manual Annotation with Mask')
plt.show()

Build CONV2D Model

In [ ]:
def build_annotator_model(input_shape, output_channels, dropout_rate=0.0):
    """
    Builds a TensorFlow model that takes original images as input and produces annotated images as output.
    
    Args:
        input_shape: The shape of the input image (e.g. (256, 256, 3) for a 256x256 RGB image).
        output_channels: The number of output channels (i.e. the number of annotation colors).
    
    Returns:
        The TensorFlow model.
    """
    # Define the input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Encoder
    conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='gelu')(inputs)
    conv1 = tf.keras.layers.Dropout(dropout_rate)(conv1)
    # conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='gelu')(conv1)
    bn1 = tf.keras.layers.BatchNormalization(name='BN1')(conv1)

    conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='gelu')(bn1)
    conv2 = tf.keras.layers.Dropout(dropout_rate)(conv2)
    # conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='gelu')(conv2)
    bn2 = tf.keras.layers.BatchNormalization(name='BN2')(conv2)
    
    conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='gelu')(bn2)
    conv3 = tf.keras.layers.Dropout(dropout_rate)(conv3)
    # conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='gelu')(conv3)
    bn3 = tf.keras.layers.BatchNormalization(name='BN3')(conv3)

    conv4 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='gelu')(bn3)
    conv4 = tf.keras.layers.Dropout(dropout_rate)(conv4)
    # conv4 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='gelu')(conv4)
    bn4 = tf.keras.layers.BatchNormalization(name='BN4')(conv4)

    conv5 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='gelu')(bn4)
    conv5 = tf.keras.layers.Dropout(dropout_rate)(conv5)
    # conv5 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='gelu')(conv5)
    bn5 = tf.keras.layers.BatchNormalization(name='BN5')(conv5)

    
    # Decoder
    deconv1 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size=3, strides=2, padding='same', activation='gelu')(bn5)
    deconv1 = tf.keras.layers.Dropout(dropout_rate)(deconv1)
    #deconv1 = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size=3, strides=2, padding='same', activation='gelu')(deconv1)
    bn6 = tf.keras.layers.BatchNormalization(name='BN6')(deconv1)

    skip1 = tf.keras.layers.concatenate([bn4, bn6])

    deconv2 = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, strides=2, padding='same', activation='gelu')(skip1)
    deconv2 = tf.keras.layers.Dropout(dropout_rate)(deconv2)
    # deconv2 = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, strides=2, padding='same', activation='gelu')(deconv2)
    bn7 = tf.keras.layers.BatchNormalization(name='BN7')(deconv2)

    skip2 = tf.keras.layers.concatenate([bn3, bn7])

    deconv3 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same', activation='gelu')(skip2)
    deconv3 = tf.keras.layers.Dropout(dropout_rate)(deconv3)
    # deconv3 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same', activation='gelu')(deconv3)
    bn8 = tf.keras.layers.BatchNormalization(name='BN8')(deconv3)

    skip3 = tf.keras.layers.concatenate([bn2, bn8])

    deconv4 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='gelu')(skip3)
    deconv4 = tf.keras.layers.Dropout(dropout_rate)(deconv4)
    # deconv4 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='gelu')(deconv4)

    bn9 = tf.keras.layers.BatchNormalization(name='BN9')(deconv4)
    skip4 = tf.keras.layers.concatenate([bn1, bn9])

    deconv5 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='gelu')(skip4)
    deconv5 = tf.keras.layers.Dropout(dropout_rate)(deconv5)
    # deconv5 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='gelu')(deconv5)
    bn10 = tf.keras.layers.BatchNormalization(name='BN10')(deconv5)
    
    segmentation_mask = tf.keras.layers.Conv2DTranspose(filters=output_channels, kernel_size=3, strides=1, padding='same', activation='linear')(bn10)
    
    # annotated_pred = tf.keras.layers.Multiply()([inputs, segmentation_mask])
    annotated_pred = tf.keras.layers.Add()([inputs, segmentation_mask])
    annotated_pred = tf.keras.layers.Multiply()([annotated_pred,(tf.ones_like(input_validation_batch[0].shape)*255.) ])
    annotated_pred = tf.keras.layers.Activation('relu')(annotated_pred)

    # outputs = tf.keras.layers.Flatten()(annotated_pred)
    # outputs = tf.keras.layers.Dense(input_shape[0], activation='softmax')(outputs)
    # outputs = tf.keras.layers.Reshape(target_shape=(input_shape[0],input_shape[1],channels))(outputs)
    
    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=annotated_pred, name='Annotation_model')
    
    return model


In [ ]:
try:
  del model
except:
  pass

# Define the input and output shapes
input_shape = (height, width, channels)
output_channels = channels

# if model:  del model
# Build the model
model = build_annotator_model(input_shape, output_channels)

# Create Optimizer.
# Define the parameters for the schedule
initial_learning_rate = 1e-2
decay_steps = 1200
first_decay_steps = 1200

# Create the schedule
optimizer = Adam(learning_rate=initial_learning_rate, amsgrad=True)
moving_avg_Adam = tfa.optimizers.MovingAverage(optimizer)
stocastic_avg_Adam = tfa.optimizers.SWA(optimizer) 

# Compile the model with an appropriate loss function and optimizer
# model.compile(loss=PixelWiseCrossEntropy(), optimizer=optimizer, metrics=['acc'], run_eagerly=True)
model.compile(loss='mse', optimizer=stocastic_avg_Adam, metrics=['mse','acc'], run_eagerly=True)


with tf.device('/device:GPU:0'):
  _ = model.predict(validation_generator, steps=1, verbose=1)
  _ = model.evaluate(train_generator, verbose=1)  
  # _ = model.fit(train_generator, epochs=1, verbose=1)  

model.summary()

Visualize Annotation Model Graph

In [ ]:
plot_model(model, show_shapes=True, to_file=os.path.join(DATA_FOLDER, f'model/Annotation_model.png'), show_layer_names=True, rankdir='TB', expand_nested=True, dpi=50)

## Implement Callback Functions

During training we want to save checkpoints and log the progress to TensorBoard so we create the appropriate callbacks for Keras.
This is the callback for writing checkpoints during training.

In [ ]:
from tensorflow.python.framework.test_util import run_functions_eagerly
save_format = 'h5'
path_checkpoint = os.path.join(DATA_FOLDER, f'model/Annotation_model.{save_format}')
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_acc',
                                      verbose=1,
                                      save_weights_only=True,
                                      restore_best_weights=True,
                                      save_best_only=True, 
                                      run_functions_eagerly=True,
                                      save_format=save_format)

path_checkpoint_MA = os.path.join(DATA_FOLDER, f'model/Annotation_model_MA.{save_format}')

path_checkpoint_SWA = os.path.join(DATA_FOLDER, f'model/Annotation_model_SWA.{save_format}')

callback_MA = tfa.callbacks.AverageModelCheckpoint(filepath=path_checkpoint_MA, 
                                                    monitor='val_acc',
                                                    update_weights=True,
                                                    save_weights_only=True,
                                                    run_functions_eagerly=True, 
                                                    save_format=save_format)

callback_SWA = tfa.callbacks.AverageModelCheckpoint(filepath=path_checkpoint_SWA,
                                                    monitor='val_acc',
                                                    update_weights=True,
                                                    save_weights_only=True,
                                                    run_functions_eagerly=True,
                                                    save_format=save_format)

This is the callback for stopping the optimization when performance worsens on the validation-set.

In [ ]:
callback_early_stopping = EarlyStopping(  monitor="val_acc",
                                          # min_delta=0,
                                          patience=400,
                                          verbose=1,
                                          mode="auto",
                                          # baseline=None,
                                          restore_best_weights=True )

This is the callback for writing the TensorBoard log during training.

In [ ]:
dirpaths = [Path(os.path.join(DATA_FOLDER, r'model//TensorBoard/'))]

for dirpath in dirpaths:
    if dirpath.exists() and dirpath.is_dir():
        try:        
            shutil.rmtree(dirpath, ignore_errors=True)
            os.chmod(dirpath, 0o777)
            os.rmdir(dirpath)
            os.removedirs(dirpath)
            print("Directory '%s' has been removed successfully", dirpath)
        except OSError as error:
            print(error)
            print("Directory '%s' can not be removed", dirpath)

logdir = os.path.join(DATA_FOLDER, r'model//TensorBoard/')
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
callback_tensorboard = TensorBoard(log_dir=logdir,
                                   histogram_freq=1,
                                   write_graph=True,
                                   profile_batch = '500,520')

This callback reduces the learning-rate for the optimizer if the validation-loss has not improved since the last epoch (as indicated by `patience=100`). The learning-rate will be reduced by multiplying it with the given factor. We set a start learning-rate of 1e-2 above, so multiplying it by 0.95 gives a learning-rate of 9.5e-3. We don't want the learning-rate to go any lower 1e-5

In [ ]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_acc',
                                       factor=0.99,
                                       min_lr=1e-5,
                                       patience=5,
                                       verbose=1)    


In [ ]:
class CosineDecayRestarts(tf.keras.callbacks.Callback):
    def __init__(self, initial_learning_rate, first_decay_steps, alpha=0.0, t_mul=2.0, m_mul=1.0):
        super(CosineDecayRestarts, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.first_decay_steps = first_decay_steps
        self.alpha = alpha
        self.t_mul = t_mul
        self.m_mul = m_mul
        self.batch_step = 0

    def on_train_batch_begin(self, step, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(self.batch_step, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        self.batch_step += 1

    def schedule(self, step, lr):
        def compute_step(completed_fraction, geometric=False):
            """Helper for `cond` operation."""
            if geometric:
                i_restart = math_ops.floor(
                  math_ops.log(1.0 - completed_fraction * (1.0 - self.t_mul)) /
                  math_ops.log(self.t_mul))

                sum_r = (1.0 - self.t_mul**i_restart) / (1.0 - self.t_mul)
                completed_fraction = (completed_fraction - sum_r) / self.t_mul**i_restart

            else:
                i_restart = math_ops.floor(completed_fraction)
                completed_fraction -= i_restart

            return i_restart, completed_fraction

        completed_fraction = step / self.first_decay_steps

        i_restart, completed_fraction = control_flow_ops.cond(
          math_ops.equal(self.t_mul, 1.0),
          lambda: compute_step(completed_fraction, geometric=False),
          lambda: compute_step(completed_fraction, geometric=True))

        m_fac = self.m_mul**i_restart
        cosine_decayed = 0.5 * m_fac * (1.0 + math_ops.cos(
          constant_op.constant(math.pi) * completed_fraction))
        decayed = (1 - self.alpha) * cosine_decayed + self.alpha

        return math_ops.multiply(self.initial_learning_rate, decayed)

callback_cosine_decay_restarts = CosineDecayRestarts( initial_learning_rate,
                                                      first_decay_steps )

In [ ]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard,
             callback_cosine_decay_restarts,
             callback_SWA,
             callback_reduce_lr]

# callbacks = list()

#### Load weights from last checkpoint

In [ ]:
def train_model(resume, epochs, initial_epoch, batch_size, model, savepath):
    with tf.device('/device:GPU:0'):
        print(model.summary())
        history=model.fit(train_generator, 
                          steps_per_epoch=steps_per_epoch, 
                          epochs=EPOCHS, 
                          verbose=1, 
                          callbacks=callbacks,
                          validation_data=validation_generator, 
                          validation_steps=validation_steps, 
                          #validation_freq=5,
                          #class_weight=None, 
                          #max_queue_size=10, 
                          #workers=8, 
                          #use_multiprocessing=True,
                          shuffle=True, 
                          initial_epoch=initial_epoch)
        
        model.load_weights(savepath)
        print("Checkpoint Loaded")  
        model.evaluate(test_generator, steps=test_steps)

    if resume:
        try:
            #del model
            model.load_weights(savepath, {"CustomLoss":CustomLoss()})
            #model = load_model(savepath, {"CustomLoss":CustomLoss()})
            print("Checkpoint Loaded")  
        except Exception as error:
            print("Error trying to load checkpoint.")
            print(error)

    return history, model
    
with tf.device('/device:GPU:0'):
    def plot_train_history(history, title):
        loss = history.history['loss']
        accuracy = history.history['acc']
        mape = history.history['mape']
        mae = history.history['mae']
        val_acc = history.history['val_acc']
        val_accuracy = history.history['val_acc']
        val_mae = history.history['val_mae']
        val_mape = history.history['val_mape']
        epochs = range(len(loss))
        plt.figure(figsize=(30,5))
        plt.plot(epochs, loss, label='training_loss') 
        plt.plot(epochs, val_acc, label='validation_loss')
        plt.show()
        plt.figure(figsize=(30,5))
        plt.plot(epochs, accuracy, label='training_accuracy') 
        plt.plot(epochs, val_accuracy, label='validation_accuracy')
        plt.show()
        plt.figure(figsize=(30,5))
        plt.plot(epochs, mae, label='training_mae') 
        plt.plot(epochs, val_mae, label='validation_mae')
        plt.show()
        plt.figure(figsize=(30,5))
        plt.plot(epochs, mape, label='training_mape') 
        plt.plot(epochs, val_mape, label='validation_mape')
        plt.show()
        return

## Train Model

# Load the TensorBoard notebook extension
# %load_ext tensorboard
%reload_ext tensorboard

%tensorboard --logdir './data/model/TensorBoard/'

In [ ]:
EPOCHS = 100000

train_steps = steps_per_epoch = len(train_image_paths)//batch_size
test_steps = validation_steps =  len(validation_image_paths)//batch_size
test_generator = validation_generator

try:
  model.load_weights(os.path.join(DATA_FOLDER, f'model/Annotation_model.{save_format}'))
  model.save_weights(os.path.join(DATA_FOLDER, f'model/Annotation_model'), save_format=save_format)
  # model.set_weights(load_model(os.path.join(DATA_FOLDER, f'model/Annotation_model'), {"CustomLoss":CustomLoss()()}).get_weights())
  # model.save(os.path.join(DATA_FOLDER, f'model/Annotation_model'), save_format=save_format)
  # model = load_model(os.path.join(DATA_FOLDER, f'model/Annotation_model.{save_format}'), {"CustomLoss":CustomLoss()}, compile=True)
  print("Checkpoint Loaded")
except Exception as error:
  print("Error trying to load checkpoint.")
  print(error)
                     
history, model = train_model(resume=False, epochs=EPOCHS, initial_epoch=0, batch_size=batch_size, model=model, savepath=path_checkpoint)

### Load Checkpoint

Because we use early-stopping when training the model, it is possible that the model's performance has worsened on the test-set for several epochs before training was stopped. We therefore reload the last saved checkpoint, which should have the best performance on the test-set.

In [ ]:
with tf.device('/device:GPU:0'):
    try:
        model.load_weights(os.path.join(DATA_FOLDER, f'model/Annotation_model.{save_format}'))
        model = load_model(os.path.join(DATA_FOLDER, f'model/Annotation_model'), {"CustomLoss":CustomLoss()})
        # model.set_weights(load_model(os.path.join(DATA_FOLDER, f'model/Annotation_model'), {"CustomLoss":CustomLoss()}).get_weights())
        print("Checkpoint Loaded")
    except Exception as error:
        print("Error trying to load checkpoint.")
        print(error)

## Performance on Test-Set

We can now evaluate the model's performance on the test-set. This function expects a batch of data, but we will just use one long time-series for the test-set, so we just expand the array-dimensionality to create a batch with that one sequence.

In [ ]:
with tf.device('/device:GPU:0'):
    model.evaluate(train_generator, steps=train_steps)
    model.evaluate(validation_generator, steps=validation_steps)
    model.evaluate(test_generator, steps=test_steps)

Predict on Test Set

In [ ]:
input_validation_batch.max()
annotated_pred.max()


In [ ]:
# Predict the annotation with model
annotated_pred = model.predict(input_validation_batch, steps=1)

for idx in range(batch_size):
    plt.imshow(input_validation_batch[idx])
    plt.title('Original Image')
    plt.show()
    plt.imshow(output_validation_batch[idx])
    plt.title('True Annotation Mask')
    plt.show()
    plt.imshow(annotated_pred[idx])
    plt.title('Predicted Annotation Mask')
    plt.show()
    plt.imshow(input_validation_batch[idx] + output_validation_batch[idx])
    plt.title('True Annotated Image')
    plt.show()
    plt.imshow(input_validation_batch[idx] + annotated_pred[idx])
    plt.title('Manual Annotation with Predicted Mask')
    plt.show()